In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
url = 'https://docs.anaconda.com/anaconda/packages/old-pkg-lists/4.0.0/py34/'
r = requests.get(url)
s = BeautifulSoup(r.content)
current = s.find(class_='current reference internal') # Gets this page's Anaconda and Python versions
anaconda_v, python_v = re.findall(r'Packages included in Anaconda ([\d\.]*\d) for Python version ([\d\.]*\d)',current.string)[0]
display(anaconda_v, python_v)

In [5]:

current

<a class="current reference internal" href="#">Packages included in Anaconda 4.0.0 for Python version 3.4</a>

In [6]:
type(current)

bs4.element.Tag

In [7]:
current.string

'Packages included in Anaconda 4.0.0 for Python version 3.4'

'4.0.0'

'3.4'